In [1]:
import pandas as pd
import numpy as np

In [2]:
b=pd.read_excel("BTAB.xlsx")
b

,Date,Open,High,Low,Close
0,2020-01-20,4382.06,4467.52,4382.06,4436.79
1,2020-01-19,4149.82,4384.80,4149.82,4382.06
2,2020-01-16,4068.20,4161.75,4066.64,4149.82
3,2020-01-15,4036.23,4119.47,4036.23,4068.20
4,2020-01-14,4123.48,4123.87,4009.70,4036.23
...,...,...,...,...,...
536,2020-01-27,4528.20,4531.45,4469.43,4493.01
537,2020-01-26,4513.89,4549.79,4503.79,4528.20
538,2020-01-23,4440.29,4519.84,4440.29,4513.89
539,2020-01-22,4408.10,4446.05,4407.28,4440.29


In [3]:
i=b['Close']
print(i)

0      4436.79
1      4382.06
2      4149.82
3      4068.20
4      4036.23
        ...   
536    4493.01
537    4528.20
538    4513.89
539    4440.29
540    4408.10
Name: Close, Length: 541, dtype: float64


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
dff=scaler.fit_transform(np.array(i).reshape(-1,1))


In [5]:
training_size=int(len(dff)*0.75)
test_size=len(dff)-training_size
train_data,test_data=dff[0:training_size,:],dff[training_size:len(dff),:1]

In [6]:
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step-1),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX),np.array(dataY)

In [7]:
time_step=100
X_train,y_train=create_dataset(train_data,time_step)
X_test,ytest=create_dataset(test_data,time_step)

In [9]:
print(X_train.shape),print(y_train.shape)

(304, 99)
(304,)


(None, None)

In [10]:
print(X_test.shape),print(ytest.shape)

(35, 99)
(35,)


(None, None)

In [11]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test.reshape(X_test.shape[0],X_test.shape[1],1)

array([[[0.73066551],
        [0.74544637],
        [0.75102051],
        ...,
        [0.062115  ],
        [0.13460462],
        [0.19379071]],

       [[0.74544637],
        [0.75102051],
        [0.76615137],
        ...,
        [0.13460462],
        [0.19379071],
        [0.23119234]],

       [[0.75102051],
        [0.76615137],
        [0.77010817],
        ...,
        [0.19379071],
        [0.23119234],
        [0.20350217]],

       ...,

       [[0.72508031],
        [0.75339312],
        [0.75886778],
        ...,
        [0.31893808],
        [0.32341065],
        [0.33408736]],

       [[0.75339312],
        [0.75886778],
        [0.78021014],
        ...,
        [0.32341065],
        [0.33408736],
        [0.32754428]],

       [[0.75886778],
        [0.78021014],
        [0.77980489],
        ...,
        [0.33408736],
        [0.32754428],
        [0.34050886]]])

In [12]:
!pip install keras

In [13]:
!pip install tensorflow

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [19]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(99,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_square_error',optimizer='adam')

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 99, 50)            10400     
                                                                 
 lstm_4 (LSTM)               (None, 99, 50)            20200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 50851 (198.64 KB)
Trainable params: 50851 (198.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100


ValueError: in user code:

    File "E:\Anaconda\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\compile_utils.py", line 252, in __call__
        self.build(y_pred)
    File "E:\Anaconda\Lib\site-packages\keras\src\engine\compile_utils.py", line 194, in build
